# Day 3 - Model Calibration

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner
from scipy.integrate import odeint
from scipy.optimize import minimize
from IPython.display import display, Math
from scipy.optimize import Bounds

## Defining the model
Let N be the number of tumor cells and assume that this number can increase with a growth rate of $r$ cells/hour until it reaches the environmental carrying capacity of $K$ cells. With these assumptions, the mathematical model is:
\begin{equation}
\frac{dN}{dt}=rN\left(1-\frac{N}{K}\right),
\end{equation}

- $\boldsymbol{\theta}$: vector of model parameters, $\boldsymbol{\theta}=(r,K)$;
- $r$: tumor growth rate;
- $K$: environmental carrying capacity;
- $\boldsymbol{Y}(\boldsymbol{\theta})$: model prediction;

In [ ]:
# Let y be the vector of outputs, t the time, and theta the vector of parameters
def control_tumor(y, t, theta):
    return theta[0] * y[0] * (1.0 - y[0] / theta[1])

## Generating the data
We will generate the in silico data using the true model and adding a random perturbation

In [ ]:
# Define time interval, parameter value, and initial condition
times = np.arange(0,50,1)
true_growth, true_ic, true_carry = 0.25, 1, 60
# Solving the ODE model
y = odeint(control_tumor, t=times, y0=true_ic, args=tuple([[true_growth,true_carry]]))

In [ ]:
#Adding perturbation
true_std = 3.6
np.random.seed(2021) # For reproducibility
yobs = np.random.normal(y,true_std)
yobs[yobs < 0] = 0

In [ ]:
# Plotting the true and observed data
fig, ax = plt.subplots(dpi=120)
plt.plot(times, y, label='True data', color='black',linewidth=2.0)
plt.scatter(times, yobs, label='Measured data', color='red')
plt.legend()
plt.xlabel('Time (hours)')
plt.ylabel('Cell number');
#plt.savefig("just_data.pdf")
plt.show()

## Maximum likelihood estimation

Find $\hat{\boldsymbol{\theta}}\in\Theta$, where $\Theta$ is the parameter space, that maximize the likelihood.\bigskip

It is customary to work with the more manageable log-likelihood function.

\begin{align*}
\hat{\boldsymbol{\theta}}&=\underset{\boldsymbol{\theta}\in\Theta}{\text{argmax}}[\log\pi(\boldsymbol{D}|\boldsymbol{\theta})];\\
&=\underset{\boldsymbol{\theta}\in\Theta}{\text{argmax}}\left[-\frac{N_t}{2}\log(2\pi)-\frac{N_t}{2}\log(\sigma^2)-\sum_{i=1}^{N_t}\frac{1}{2}\frac{(D_{i}-Y_i(\boldsymbol{\theta}))^2}{2\sigma^2}\right].
\end{align*}

In [ ]:
def log_likelihood(theta, times, y):
    cal_growth, cal_carry, cal_std = theta
    model = odeint(control_tumor, t=times, y0=y[0], args=tuple([[cal_growth,cal_carry]]))
    variance = cal_std*cal_std
    return -0.5 * np.sum((y - model) ** 2 / variance + np.log(2*np.pi) + np.log(variance))

In [ ]:
nll = lambda *args: -log_likelihood(*args)
initial = np.array([0.4, 50.0, 0.3])+ 0.1 * np.random.randn(3)
bounds = Bounds([0.0, 10.0, 0.1], [1.0, 120.0, 50.0])
soln = minimize(nll, initial, args=(times, yobs), bounds=bounds)
ml_growth, ml_carry, ml_std = soln.x

print("Maximum likelihood estimates:")
print("Growth rate = %f" % ml_growth)
print("Carrying capacity = %f" % ml_carry)
print("Standard deviation = %f" % ml_std)

error_g=100*abs(ml_growth-true_growth)/true_growth
error_k=100*abs(ml_carry-true_carry)/true_carry
error_s=100*abs(ml_std-true_std)/true_std

print("Growth rate %% error = %f" % error_g)
print("Carrying capacity %% error = %f" % error_k)
print("Standard deviation %% error = %f" % error_s)

In [ ]:
y_ml = odeint(control_tumor, t=times, y0=yobs[0], args=tuple([[ml_growth,ml_carry]]))
fig, ax = plt.subplots(dpi=120)
plt.plot(times, y_ml, label='Calibrated model', linestyle='dashed', color='blue',linewidth=2.0)
plt.plot(times, y, label='True data', color='black', alpha=0.75,linewidth=2.0)
plt.scatter(times, yobs, label='Measured data', color='red')
plt.legend()
plt.xlabel('Time (hours)')
plt.ylabel('Cell number');
#plt.savefig("ml_fit.pdf")
plt.show()

In [ ]:
def log_likelihood_ic(theta, times, y):
    cal_growth, cal_carry, cal_std, cal_ic = theta
    model = odeint(control_tumor, t=times, y0=cal_ic, args=tuple([[cal_growth,cal_carry]]))
    variance = cal_std*cal_std
    return -0.5 * np.sum((y - model) ** 2 / variance + np.log(2*np.pi) + np.log(variance))

In [ ]:
nll = lambda *args: -log_likelihood_ic(*args)
initial = np.array([0.4, 50.0, 0.3, 5.0])+ 0.1 * np.random.randn(4)
bounds = Bounds([0.0, 10.0, 0.1, 0.1], [1.0, 120.0, 50.0, 20.0])
soln = minimize(nll, initial, args=(times, yobs), bounds=bounds)
ml_growth, ml_carry, ml_std, ml_ic = soln.x

print("Maximum likelihood estimates:")
print("Growth rate = %f" % ml_growth)
print("Carrying capacity = %f" % ml_carry)
print("Standard deviation = %f" % ml_std)
print("Initial condition = %f" % ml_ic)

error_g=100*abs(ml_growth-true_growth)/true_growth
error_k=100*abs(ml_carry-true_carry)/true_carry
error_s=100*abs(ml_std-true_std)/true_std
error_i=100*abs(ml_ic-true_ic)/true_ic

print("Growth rate %% error = %f" % error_g)
print("Carrying capacity %% error = %f" % error_k)
print("Standard deviation %% error = %f" % error_s)
print("Initial condition %% error = %f" % error_i)

In [ ]:
y_ml = odeint(control_tumor, t=times, y0=ml_ic, args=tuple([[ml_growth,ml_carry]]))
fig, ax = plt.subplots(dpi=120)
plt.plot(times, y_ml, label='Calibrated model', linestyle='dashed', color='blue',linewidth=2.0)
plt.plot(times, y, label='True data', color='black', alpha=0.75,linewidth=2.0)
plt.scatter(times, yobs, label='Measured data', color='red')
plt.legend()
plt.xlabel('Time (hours)')
plt.ylabel('Cell number');
#plt.savefig("ml_fit.pdf")
plt.show()

## Bayesian approach
Given events A and B:
\begin{align*}
P(A,B)&=P(A|B)P(B);\\
P(A,B)&=P(B,A);\\
P(B,A)&=P(B|A)P(A);\\
P(A|B)P(B)&=P(B|A)P(A);\\
P(A|B)&=\frac{P(B|A)P(A)}{P(B)};
\end{align*}

Converting to probability densities $\pi$, if $A$ represents the parameter $\boldsymbol{\theta}$ of a model, and $B$ the observational data $\boldsymbol{D}$:
\begin{align*}
\underbrace{\pi(\boldsymbol{\theta}|\boldsymbol{D})}_{\text{posterior}}&=\frac{\overbrace{\pi(\boldsymbol{D}|\boldsymbol{\theta})}^{\text{likelihood}}\overbrace{\pi(\boldsymbol{\theta})}^{\text{prior}}}{\underbrace{\pi(\boldsymbol{D})}_{\text{evidence}}};\qquad \pi(\boldsymbol{D})=\int_\Theta\pi(\boldsymbol{D}|\boldsymbol{\theta})\pi(\boldsymbol{\theta})\, \text{d}\theta\\
\pi(\boldsymbol{\theta}|\boldsymbol{D})&\propto\pi(\boldsymbol{D}|\boldsymbol{\theta})\pi(\boldsymbol{\theta});\\
\log\pi(\boldsymbol{\theta}|\boldsymbol{D})&\propto\log\pi(\boldsymbol{D}|\boldsymbol{\theta})+\log\pi(\boldsymbol{\theta});
\end{align*}

In [ ]:
def log_prior(theta):
    cal_growth, cal_carry, cal_std, cal_ic = theta
    if 0.0 < cal_growth < 1.0 and 10.0 < cal_carry < 120.0 and 0.1 < cal_std < 50.0 and 0.1 < cal_ic < 20.0:
        return 0.0
    return -np.inf

In [ ]:
def log_probability(theta, times, y):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood_ic(theta, times, y)

In [ ]:
mean_g = np.array([0.4, 50.0, 0.3, 10.0])
pos = mean_g + 1e-2 * np.random.randn(10,4)

In [ ]:
nwalkers, ndim = pos.shape
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(times, yobs))
sampler.run_mcmc(pos, 12500, progress=True);

In [ ]:
fig, axes = plt.subplots(4, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["Growth rate", "Carrying Capacity", "Standard deviation", "Initial condition"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
tau = sampler.get_autocorr_time()
print(tau)

In [ ]:
flat_samples = sampler.get_chain(discard=450, flat=True)
print(flat_samples.shape)

In [ ]:
fig = corner.corner(flat_samples, labels=labels, truths=[true_growth, true_carry, true_std, true_ic]);

In [ ]:
fig, ax = plt.subplots(dpi=120)
inds = np.random.randint(len(flat_samples), size=100)
for ind in inds:
    sample = flat_samples[ind]
    y_sp = odeint(control_tumor, t=times, y0=sample[3], args=tuple([[sample[0],sample[1]]]))
    plt.plot(times, y_sp, 'blue', alpha=0.5)
plt.plot(times, y_sp, label='Calibrated model', color='blue', alpha=0.5)
plt.scatter(times, yobs, zorder=100, label='Measured data', color='red')
plt.legend()
plt.xlabel('Time (hours)')
plt.ylabel('Cell number');
#plt.savefig("bc_fit.pdf")
plt.show()

In [ ]:
for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{{2:.3f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels[i])
    display(Math(txt))